In [1]:
def qbratings(year):
    import pandas as pd
    url = f'https://www.pro-football-reference.com/years/{year}/passing.htm'
    df = pd.read_html(url,header=0)[0]
    df = df[df['QBrec'].notna()]
    df = df[['Player','Tm','Pos','Age','G','GS','Cmp','Att','Yds','TD','Int','Rate']]
    df = df[df['Pos'] != 'Pos']
    df = df.replace('\+','',regex=True).astype(object)
    df = df.replace('\*','',regex=True).astype(object)
    df = df.replace('^qb','QB',regex=True).astype(object)
    df = df.replace('^/qb','QB',regex=True).astype(object)
    df = df.replace('^wr','QB',regex=True).astype(object)
    df['Pos'].fillna('QB',inplace=True)
    df['Player'] = df['Player'].str.rstrip()
    columns = df.columns[3:]
    for col in columns:
        df[col] = pd.to_numeric(df[col],errors='coerce')
    df['Year'] = year
    df['Player'] = df.Player.str.replace('[^a-zA-Z]', '', regex = True)
    playerID = []
    for index, row in df.iterrows():
        playerID.append(row['Player'] + str(row['Year']))
    df['PlID'] = playerID
    teamID = []
    for index, row in df.iterrows():
        teamID.append(row['Tm'] + str(row['Year']))
    df['TmID'] = teamID
    return df

In [2]:
def createQBDB(startYear,endYear,function):
    # Import dependencies
    import pandas as pd
    database = pd.DataFrame()
    years = []
    # Iterates through years specified
    for year in range(startYear, endYear + 1):
        years.append(year)
    for year in years:
        database = database.append(function(year))
    return database

In [3]:
df = createQBDB(2000,2021,qbratings)

In [4]:
import pandas as pd
years = range(2000,2022)
columns = ['Cmp','Att','Yds','TD','Int']
players = []
team = []
cmp = []
att = []
yds = []
td = []
inter = []
rate = []
yrs = []
for index, row in df.iterrows():
    player = row['Player']
    yr = row['Year']
    tm = row['Tm']
    players.append(player)
    yrs.append(yr)
    team.append(tm)
    for year in years:
        if year == yr:
            for column in columns:
                amt = df.loc[(df['Player'] == player) & (df['Year'] <= yr), column].sum()
                if column == 'Cmp':
                    cmp.append(amt)
                elif column == 'Att':
                    att.append(amt)
                elif column == 'Yds':
                    yds.append(amt)
                elif column == 'TD':
                    td.append(amt)
                else:
                    inter.append(amt)
                mean = round(df.loc[(df['Player'] == player) & (df['Year'] <= yr), 'Rate'].mean(),2)
            rate.append(mean)
        else:
            continue
newdf = pd.DataFrame({'Player':players,'Team':team,'Cmp':cmp,'Att':att,'Yds':yds,'TD':td,'Int':inter,'AvgRate':rate,'Year':yrs})

In [5]:
newdf['PassRate'] = ((((((newdf['Cmp']/newdf['Att']) - .3)*5)+
                       (((newdf['Yds']/newdf['Att'])-3)*.25)+
                       ((newdf['TD']/newdf['Att'])*20)+
                       (2.375-((newdf['Int']/newdf['Att'])*25)))/6)*100)

playerID = []
for index, row in newdf.iterrows():
    playerID.append(row['Player'] + str(row['Year']))

newdf['PlID'] = playerID

In [6]:
columns = ['Player','Team','Cmp','Att','Yds','TD','Int','AvgRate','PassRate','Year','PlID']
newdf = newdf[columns]
newdf

,Player,Team,Cmp,Att,Yds,TD,Int,AvgRate,PassRate,Year,PlID
0,PeytonManning,IND,357,571,4413,33,15,94.70,94.705925,2000,PeytonManning2000
1,JeffGarcia,SFO,355,561,4278,31,10,97.60,97.582442,2000,JeffGarcia2000
2,ElvisGrbac,KAN,326,547,4169,28,14,89.90,89.903260,2000,ElvisGrbac2000
3,DaunteCulpepper,MIN,297,474,3937,33,16,98.00,98.048523,2000,DaunteCulpepper2000
4,BrettFavre,GNB,338,580,3812,20,16,78.00,78.031609,2000,BrettFavre2000
...,...,...,...,...,...,...,...,...,...,...,...
1218,SeanMannion,MIN,56,94,500,1,2,63.90,68.572695,2021,SeanMannion2021
1219,BrandonAllen,CIN,146,260,1589,10,6,77.30,77.548077,2021,BrandonAllen2021
1220,NickMullens,CLE,407,630,4861,26,22,88.03,87.275132,2021,NickMullens2021
1221,IanBook,NOR,12,20,135,0,2,40.60,38.541667,2021,IanBook2021


In [7]:
newdf.to_csv('../Database_CSVs/passerratings.csv',index=False)